In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create the vertices/nodes, edges ..


In [ ]:

#  import pandas as pd
#  
#  print("--")



In [ ]:

#  A larger set of Airports
#

df_airports = pd.DataFrame([
   ["MKE", "Milwaukee"               ,"Airport"],
   ["ORD", "Chicago O-Hare"          ,"Airport"],
   ["SJC", "San Jose"                ,"Airport"],
   ["DEN", "Denver"                  ,"Airport"],
      #
   ["MSP", "Minneapolis, St.Paul"    ,"Airport"],
   ["MSN", "Madison"                 ,"Airport"],
   ["GRB", "Green Bay"               ,"Airport"],
      #
   ["EGE", "Eagle, Vail"             ,"Airport"],
   ["COS", "Colorado Springs"        ,"Airport"],
   ["PUB", "Pueblo"                  ,"Airport"],
   ["ASE", "Aspen"                   ,"Airport"],
   ["FNL", "Fort Collins, Loveland"  ,"Airport"],
      #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


In [10]:

#  Create the edge, flights ..
#

df_flights = pd.DataFrame([
   ["MKE", "ORD",   66, 1, "FLIES_TO"],
   ["ORD", "MKE",   66, 1, "FLIES_TO"],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO"],
   ["DEN", "ORD",  886, 1, "FLIES_TO"],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO"],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO"],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO"],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
      #
   ["MSN", "ORD",  135, 1, "FLIES_TO"],                           #  ORD <-->  MSN, MSP, GRB
   ["ORD", "MSN",  135, 1, "FLIES_TO"],
   ["MSP", "ORD",  334, 1, "FLIES_TO"],
   ["ORD", "MSP",  334, 1, "FLIES_TO"],
   ["GRB", "ORD",  196, 1, "FLIES_TO"],
   ["ORD", "GRB",  196, 1, "FLIES_TO"],
      #
   ["EGE", "DEN",  155, 1, "FLIES_TO"],                           #  DEN <-->  EGE, COS, PUB, ASE, FNL
   ["DEN", "EGE",  155, 1, "FLIES_TO"],
   ["COS", "DEN",   95, 1, "FLIES_TO"],
   ["DEN", "COS",   95, 1, "FLIES_TO"],
   ["PUB", "DEN",  133, 1, "FLIES_TO"],
   ["DEN", "PUB",  133, 1, "FLIES_TO"],
   ["ASE", "DEN",  125, 1, "FLIES_TO"],
   ["DEN", "ASE",  125, 1, "FLIES_TO"],
   ["FNL", "DEN",   58, 1, "BUSES_TO"],
   ["DEN", "FNL",   58, 1, "BUSES_TO"],
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
|  7 | MSN        | ORD      |        135 |          1 | FLIES_TO |
|  8 | ORD        | MSN      |        135 |          1 | FLIES_TO |
|  9 | MSP        | ORD      |        334 |          1 | FLIES_TO |
| 10 | ORD        | MSP      |        334 |          1 | FLIES_TO |
| 11 | GRB        | ORD      |        196 |     

In [11]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [12]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name         | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------------+----------|
|  0 |               0 | ['Airport'] | Airport   | MKE              | Milwaukee              | node     |
|  1 |               1 | ['Airport'] | Airport   | ORD              | Chicago O-Hare         | node     |
|  2 |               2 | ['Airport'] | Airport   | SJC              | San Jose               | node     |
|  3 |               6 | ['Airport'] | Airport   | GRB              | Green Bay              | node     |
|  4 |               7 | ['Airport'] | Airport   | EGE              | Eagle, Vail            | node     |
|  5 |               8 | ['Airport'] | Airport   | COS              | Colorado Springs       | node     |
|  6 |               3 | ['Airport'] | Airport

          0/? [?op/s]

+----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+
|    |   r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   r.start.id | r.start.labels   | r.type   |
|----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------|
|  0 |          1 | ['Airport']    |               6 | FLIES_TO  |          196 |            1 | FLIES_TO |            6 | ['Airport']      | edge     |
|  1 |          3 | ['Airport']    |              10 | FLIES_TO  |          155 |            1 | FLIES_TO |            7 | ['Airport']      | edge     |
|  2 |          3 | ['Airport']    |              11 | FLIES_TO  |           95 |            1 | FLIES_TO |            8 | ['Airport']      | edge     |
|  3 |          3 | ['Airport']    |              12 | FLIES_TO  |          133 | 

In [14]:

#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN ID(n) AS FROM, ID(m) AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS, TYPE(r) AS my_type, LABELS(n) AS FROM_LABELS, LABELS(m) AS TO_LABELS
   ORDER BY FROM, TO
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+--------+------+------------+------------+-----------+---------------+-------------+
|    |   FROM |   TO |   DISTANCE |   NUM_HOPS | my_type   | FROM_LABELS   | TO_LABELS   |
|----+--------+------+------------+------------+-----------+---------------+-------------|
|  0 |      0 |    1 |         66 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  1 |      1 |    0 |         66 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  2 |      1 |    3 |        886 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  3 |      1 |    4 |        334 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  4 |      1 |    5 |        135 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  5 |      1 |    6 |        196 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  6 |      2 |    1 |       1829 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |
|  7 |      2 |    3 |        948 |          1 | FLIES_TO  | ['Airport']   | ['Airport'] |